# HW3.  Database Design

### Library Relations

- Member = {<span style="text-decoration:underline">memberID</span>, name, birthday}
- Borrow = {<span style="text-decoration:underline">borrowID</span><span style="text-decoration:underline">memberID</span>,<span style="text-decoration:underline">itemID</span>}
- BorrowTransactions = {<span style="text-decoration:underline">borrowID</span>, borrowDate, returnDate}
- Owes = {<span style="text-decoration:underline">fineID</span><span style="text-decoration:underline">memberID</span>,<span style="text-decoration:underline">itemID</span>}
- Fines = {<span style="text-decoration:underline">fineID</span>, status, amount}
- Item = {<span style="text-decoration:underline">itemID</span>, name, author, genre, status}
- History = {<span style="text-decoration:underline">itemID</span>, <span style="text-decoration:underline">recordID</span>}
- Records = {<span style="text-decoration:underline">recordID</span>, status, lastUpdated}
- Volunteers = {<span style="text-decoration:underline">memberID</span>,<span style="text-decoration:underline">staffID</span>}
- Works = {<span style="text-decoration:underline">memberID</span>,<span style="text-decoration:underline">staffID</span>}
- Staff = {<span style="text-decoration:underline">staffID</span>, name, position, employmentDate, wage}
- Hold = {{<span style="text-decoration:underline">staffID</span>,<span style="text-decoration:underline">eventID</span>}}
- Attends = {<span style="text-decoration:underline">memberID</span>, <span style="text-decoration:underline">eventID</span>}
- Events = {<span style="text-decoration:underline">eventID</span>, name, scheduledTime, scheduledDate, targetAudience}
- Located = {<span style="text-decoration:underline">eventID</span>, <span style="text-decoration:underline">roomNum</span>}
- Room = {<span style="text-decoration:underline">roomNum</span>, maxCap}

#### Creating Database

In [1]:
%load_ext sql

In [2]:
%sql sqlite:///library.db

Connecting to 'sqlite:///library.db'

In [3]:
%%sql

CREATE TABLE Member (
    memberID INTEGER PRIMARY KEY,
    name VARCHAR(100),
    birthday DATE,
    status VARCHAR(8) CHECK (status IN ('Active', 'Inactive'))
);

CREATE TABLE Borrow (
    borrowID INTEGER PRIMARY KEY,
    memberID INTEGER,
    itemID INTEGER,
    FOREIGN KEY (borrowID) REFERENCES BorrowTransactions(borrowID),
    FOREIGN KEY (memberID) REFERENCES Member(memberID),
    FOREIGN KEY (itemID) REFERENCES Item(itemID)
);


CREATE TABLE BorrowTransactions (
    borrowID INTEGER PRIMARY KEY,
    borrowDate DATE,
    returnDate DATE
);

CREATE TABLE Owes (
    fineID INTEGER PRIMARY KEY,
    memberID INTEGER,
    itemID INTEGER,
    FOREIGN KEY (fineID) REFERENCES Fines(fineID),
    FOREIGN KEY (memberID) REFERENCES Member(memberID),
    FOREIGN KEY (itemID) REFERENCES Item(itemID)
);

CREATE TABLE Fines (
    fineID INTEGER PRIMARY KEY,
    status VARCHAR(6) CHECK (status IN ('Paid', 'Unpaid')),
    amount DECIMAL(10,2) CHECK (amount >= 0)
);

CREATE TABLE Item (
    itemID INTEGER PRIMARY KEY,
    name VARCHAR(255),
    author VARCHAR(100),
    category VARCHAR(50),
    genre VARCHAR(50),
    status VARCHAR(11) CHECK (status IN ('Available', 'Unavailable'))
);

CREATE TABLE History (
    itemID INTEGER,
    recordID INTEGER,
    PRIMARY KEY (itemID, recordID),
    FOREIGN KEY (itemID) REFERENCES Item(itemID),
    FOREIGN KEY (recordID) REFERENCES Records(recordID)
);

CREATE TABLE Records (
    recordID INTEGER PRIMARY KEY,
    status VARCHAR(20) CHECK (status IN ('In System', 'Awaiting Approval', 'Archived')),
    lastUpdated DATE
);

CREATE TABLE Staff (
    memberID INTEGER,
    staffID INTEGER PRIMARY KEY,
    position VARCHAR(50),
    wage DECIMAL(10,2),
    employmentDate DATE,
    FOREIGN KEY (memberID) REFERENCES Member(memberID)
);

CREATE TABLE Volunteer (
    memberID INTEGER,
    volunteerID INTEGER PRIMARY KEY,
    employmentDate DATE,
    FOREIGN KEY (memberID) REFERENCES Member(memberID)
);

CREATE TABLE Hold (
    staffID INTEGER,
    eventID INTEGER,
    PRIMARY KEY (staffID, eventID),
    FOREIGN KEY (staffID) REFERENCES Staff(staffID),
    FOREIGN KEY (eventID) REFERENCES Events(eventID)
);

CREATE TABLE Events (
    eventID INTEGER PRIMARY KEY,
    name VARCHAR(100),
    scheduledTime TIME,
    scheduledDate DATE,
    targetAudience VARCHAR(100)
);

CREATE TABLE Located (
    eventID INTEGER,
    roomNum INTEGER,
    PRIMARY KEY (eventID, roomNum),
    FOREIGN KEY (eventID) REFERENCES Events(eventID),
    FOREIGN KEY (roomNum) REFERENCES Room(roomNum)
);

CREATE TABLE Room (
    roomNum INTEGER PRIMARY KEY,
    maxCap INTEGER
);

Running query in 'sqlite:///library.db'

++
||
++
++

#### Triggers

In [4]:
%%sql
CREATE TRIGGER update_fine_status
AFTER UPDATE ON Fines
FOR EACH ROW
BEGIN
    -- Automatically update fine status when the fine amount is paid
    UPDATE Fines
    SET status = 'Paid'
    WHERE fineID = NEW.fineID AND NEW.amount = 0;
END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [5]:
%%sql
CREATE TRIGGER generate_late_fine
AFTER INSERT ON BorrowTransactions
FOR EACH ROW
BEGIN
    -- Check if the return date is past the due date and generate a fine
    INSERT INTO Fines (memberID, itemID, status, amount)
    SELECT NEW.memberID, NEW.itemID, 'Unpaid', 10.00
    WHERE NEW.returnDate > (SELECT dueDate FROM Borrow WHERE borrowID = NEW.borrowID);
END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [6]:
%%sql
CREATE TRIGGER prevent_borrow_if_fine_unpaid
BEFORE INSERT ON Borrow
FOR EACH ROW
BEGIN
    -- Check if the member has an unpaid fine
    SELECT RAISE(ABORT, 'Cannot borrow item. Member has unpaid fines.')
    FROM Fines
    WHERE memberID = NEW.memberID AND status = 'Unpaid'
    LIMIT 1;
END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [7]:
%%sql
CREATE TRIGGER auto_archive_record
AFTER UPDATE ON Records
FOR EACH ROW
BEGIN
    -- Automatically archive record when status changes to 'Archived'
    SELECT RAISE(ABORT, 'Archiving record') 
    WHERE NEW.status = 'Archived' AND NOT EXISTS (SELECT 1 FROM ArchivedRecords WHERE recordID = NEW.recordID);

    -- Insert the record into the ArchivedRecords table
    INSERT INTO ArchivedRecords (recordID, lastUpdated)
    VALUES (NEW.recordID, CURRENT_DATE);
END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [8]:
%%sql
CREATE TRIGGER prevent_item_deletion_if_borrowed
BEFORE DELETE ON Item
FOR EACH ROW
BEGIN
    -- Prevent item deletion if it is still borrowed
    SELECT RAISE(ABORT, 'Item cannot be deleted as it is currently borrowed.')
    WHERE EXISTS (SELECT 1 FROM Borrow WHERE itemID = OLD.itemID);
END;


Running query in 'sqlite:///library.db'

++
||
++
++

#### Indexes

In [9]:
%%sql

CREATE INDEX ItemIndex ON Item(name, author);

Running query in 'sqlite:///library.db'

++
||
++
++

#### FOR TESTING PURPOSES
Delete tables if you need to

In [ ]:
%%sql 
-- Disable foreign key checks temporarily
PRAGMA foreign_keys = OFF;

-- Drop all tables (starting with those with foreign keys)
DROP TABLE IF EXISTS Hold;
DROP TABLE IF EXISTS Located;
DROP TABLE IF EXISTS Events;
DROP TABLE IF EXISTS Staff;
DROP TABLE IF EXISTS Volunteer;
DROP TABLE IF EXISTS Borrow;
DROP TABLE IF EXISTS BorrowTransactions;
DROP TABLE IF EXISTS History;
DROP TABLE IF EXISTS Records;
DROP TABLE IF EXISTS Fines;
DROP TABLE IF EXISTS Owes;
DROP TABLE IF EXISTS Item;
DROP TABLE IF EXISTS Member;
DROP TABLE IF EXISTS Room;

-- Re-enable foreign key checks
PRAGMA foreign_keys = ON;
